In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
import h5py
import os
import pandas as pd

In [ ]:
tf.compat.v1.disable_eager_execution()
os.chdir('/content/drive/My Drive/github/NNanobody/')

In [ ]:
amino=np.asarray(['I', 'L', 'V', 'F', 'M', 'C', 'A', 'G', 'P', 'T', 'S', 'Y', 'W', 'Q', 'N', 'H', 'E', 'D', 'K', 'R', 'X'])

In [ ]:
def mat2oh(datain,trim1):
  orig_seq=[]
  trims=np.bool_(trim1)
  onehot=datain.reshape(datain.shape[0],datain.shape[1],datain.shape[3])
  colrank=np.argsort(onehot,axis=1)
  colind=np.argmax(onehot,axis=1)
  for i in range(colind.shape[0]):
    temp=colind[i][trims[i]]
    temp[onehot[i][0][trims[i]]==0.05]=20
    sequence=''.join(amino[temp])
    orig_seq.append(sequence)
  orig_seq=np.asarray(orig_seq)
  return orig_seq

In [ ]:
steps = [0.005, 0.0005, 0.001, 5e-05, 0.0001] 
iterations = [10, 20, 30, 40, 50, 60, 70] 
datasets = ['Full Regression', 'Hold out Regression', 'Hold out Top 4%']
models = ['seq_32_32', 'seq_32x1_16']

In [ ]:
fi = h5py.File('./data/seeds/seeds_embedded.h5.batch1', 'r')
input_all = np.asarray(fi['data'])

In [ ]:
for dataset in datasets:
  for model_name in models:
    model = keras.models.load_model(f'./weights/regression/{dataset}/{model_name}')
    layer_input = model.input
    layer_size = model.input_shape
    output_layer = model.output


    enrichment = K.sum(output_layer[:,0])
    enrichment_all = output_layer[:,0]
    calculate_gradient = K.function([layer_input], K.gradients(enrichment, layer_input)[0])
    calculate_enrichment = K.function([layer_input], enrichment_all)

    for step in steps:
      for iteration in iterations:
        outname = f'{dataset}_{model_name}_{step}_{iteration}.tsv'
        datain = np.copy(input_all)
        continuous_data = np.copy(input_all)
        buffer_mask = np.zeros(datain.shape[0]).astype(bool)
        amino_acid_mask = np.sum(datain.reshape(datain.shape[0], datain.shape[1], datain.shape[3]), axis=1) != 0
        padding_mask = np.tile(~amino_acid_mask,(1,20)).reshape(continuous_data.shape)
        original_sequences = np.asarray(mat2oh(datain, amino_acid_mask))

        improved_padding_masks = np.empty((0, amino_acid_mask.shape[1]))
        improved_original_sequences = np.empty((0))
        improved_original_enrichment = np.empty((0))
        improved_new_sequences = np.empty((0, datain.shape[1], datain.shape[2], datain.shape[3]))
        improved_new_enrichment = np.empty((0))
        best_enrichment = np.ones(continuous_data.shape[0]) * -1.0
        buffer = np.zeros((datain.shape[0]))
        count = 0
        lr = step

        enrichment_init = calculate_enrichment([continuous_data])
        print('Initial Enrichment:', np.mean(enrichment_init))


        while True:
          for _ in range(iteration):
            gradient = calculate_gradient([continuous_data])
            gradient[buffer_mask,:,:,:] = 0
            continuous_data += gradient * lr
            if count > 100:
              lr = max(step*(count-100)**(-0.2), 1e-6)
            continuous_data[padding_mask] = 0.0
          
          onehot_continuous = continuous_data.reshape(continuous_data.shape[0],continuous_data.shape[1],continuous_data.shape[3])
          onehot_encoded = np.zeros(onehot_continuous.shape)

          amino_acid_ranked = np.argsort(onehot_continuous, axis=1) #amino acid ranked
          amino_acid_location = np.argmax(onehot_continuous, axis=1) #amino acid location

          for x in range(onehot_continuous.shape[0]):
            for y in range(onehot_continuous.shape[2]):
              if (amino[amino_acid_location[x, y]] == 'N' or amino[amino_acid_location[x, y]] == 'C'):
                amino_acid_location[x, y] = amino_acid_ranked[x, -2, y] # 2nd to last row (2nd best gradient)
              onehot_encoded[x, amino_acid_location[x, y], y] = 1.0 # assign 1 to maximum gradient AA

          onehot_encoded = onehot_encoded.reshape(onehot_encoded.shape[0], layer_size[1], layer_size[2], layer_size[3])
          onehot_encoded[padding_mask] = 0.0

          onehot_enrichment = calculate_enrichment([onehot_encoded])
          # print('Updated enrichment:', np.mean(onehot_enrichment))

          onehot_enrichment_temp = np.copy(onehot_enrichment)
          onehot_enrichment_temp[buffer_mask] = -10000.0
          bool_improved_sequences = (onehot_enrichment_temp > best_enrichment)

          if sum(bool_improved_sequences) > 0:
            best_enrichment[bool_improved_sequences] = onehot_enrichment[bool_improved_sequences]
            improved_original_sequences = np.append(improved_original_sequences, original_sequences[bool_improved_sequences], axis=0)
            improved_original_enrichment = np.append(improved_original_enrichment, enrichment_init[bool_improved_sequences], axis=0)
            improved_new_sequences = np.append(improved_new_sequences, onehot_encoded[bool_improved_sequences,:,:,:], axis=0)
            improved_new_enrichment = np.append(improved_new_enrichment, onehot_enrichment[bool_improved_sequences], axis=0)
            improved_padding_masks = np.append(improved_padding_masks, amino_acid_mask[bool_improved_sequences,:], axis=0)

          buffer[bool_improved_sequences] = 0
          buffer[~bool_improved_sequences] = buffer[~bool_improved_sequences] + 1
          buffer_mask = (buffer >= 10)

          if (sum(buffer_mask) == datain.shape[0] or count > 1000):
            print(f'Converged after {count} iterations')
            print(f'Best enrichment: {np.mean(onehot_enrichment)}')
            return_sequences = mat2oh(improved_new_sequences, improved_padding_masks)
            break

          count += 1

        tsv = np.concatenate((np.arange(return_sequences.shape[0]).reshape(return_sequences.shape[0],1),
                      return_sequences.reshape(return_sequences.shape[0],1),
                      improved_original_sequences.reshape(improved_original_sequences.shape[0],1),
                      improved_new_enrichment.reshape(improved_new_enrichment.shape[0],1),
                      improved_original_enrichment.reshape(improved_original_enrichment.shape[0],1)),
                     axis=1)

        np.savetxt(f'./generated/{model_name}/{outname}', tsv, fmt='%s', delimiter='\t')
        print(f'Saved {outname}')

Initial Enrichment: -0.3361355
Converged after 61 iterations
Best enrichment: -0.29232311248779297
Saved Full Regression_seq_32_32_0.005_10.tsv
Initial Enrichment: -0.3361355
Converged after 67 iterations
Best enrichment: 5.276655197143555
Saved Full Regression_seq_32_32_0.005_20.tsv
Initial Enrichment: -0.3361355
Converged after 56 iterations
Best enrichment: 7.331464767456055
Saved Full Regression_seq_32_32_0.005_30.tsv
Initial Enrichment: -0.3361355
Converged after 53 iterations
Best enrichment: 7.386279106140137
Saved Full Regression_seq_32_32_0.005_40.tsv
Initial Enrichment: -0.3361355
Converged after 45 iterations
Best enrichment: 7.411737442016602
Saved Full Regression_seq_32_32_0.005_50.tsv
Initial Enrichment: -0.3361355
Converged after 43 iterations
Best enrichment: 7.428102016448975
Saved Full Regression_seq_32_32_0.005_60.tsv
Initial Enrichment: -0.3361355
Converged after 38 iterations
Best enrichment: 7.437199115753174
Saved Full Regression_seq_32_32_0.005_70.tsv
Initial En